In [1]:
import numpy as np
from matplotlib import pyplot
from scipy import linalg
import copy
import iMPS
from scipy.sparse import linalg
import funcs
from scipy.optimize import bisect
import sympy 
np.set_printoptions(linewidth=110)

In [24]:
def one_svd(A):
    #   |                    |
    #---A---     --->   -u0--A1-- 
    #   |                    |
    
    shape = np.array(A.shape)
    A0 = A.reshape([shape[0],-1])
    u0,s,v = np.linalg.svd(A0)
    d = len(s>1e-10)
    A1 = np.diag(s[:d])@v[:d,:]      
    shape[0] = d
    A1 = A1.reshape(shape) 
    return u0[:,:d],A1,A1.shape
                    
def four_svd(A):
    #   |                    |
    #---A---     --->        u2
    #   |               -u0--s--u1-- 
    #                        u3
    #                        |
    #for real A
    u0,A1,_ = one_svd(A)
    A1 = A1.transpose([1,2,3,0])
    u1,A2,_ = one_svd(A1)
    A2 = A2.transpose([1,2,3,0])
    u2,A3,_ = one_svd(A2)
    A3 = A3.transpose([1,2,3,0])
    u3,A4,_ = one_svd(A3)
    s = A4.transpose([1,2,3,0])
    
    return s,u0,u1,u2,u3

In [25]:
X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])
Id = np.array([[1,0],[0,1]])
beta = 0
la=1
lb=10
HA = -la*np.kron(X,np.kron(X,np.kron(X,X)))
HB = -lb*np.kron(Z,np.kron(Z,np.kron(Z,Z)))

ZA = linalg.expm(-beta*HA)
ZB = linalg.expm(-beta*HB)
ZA = ZA/np.linalg.norm(ZA)
ZB = ZB/np.linalg.norm(ZB)


In [7]:
ZA = ZA.reshape([2]*8)
ZA = np.transpose(ZA,[0,4,1,5,2,6,3,7])
ZA = ZA.reshape([4]*4)

ZB = ZB.reshape([2]*8)
ZB = np.transpose(ZB,[0,4,1,5,2,6,3,7])
ZB = ZB.reshape([4]*4)



In [8]:
dZ = np.kron(Z,Z)
IdZ = np.kron(Id,Z)
IdX = np.kron(Id,X)
dX = np.kron(X,X)
dId = np.kron(Id,Id)
ZA1 = np.tensordot(IdX,ZA,([1],[0]))
ZA1 = np.tensordot(IdX,ZA1,([1],[0]))
ZA1 = np.tensordot(dId,ZA1,([1],[0]))
ZA1 = np.tensordot(dId,ZA1,([1],[0]))

ZB1 = np.tensordot(IdZ,ZB,([1],[0]))
ZB1 = np.tensordot(IdZ,ZB1,([1],[0]))
ZB1 = np.tensordot(IdZ,ZB1,([1],[0]))
ZB1 = np.tensordot(IdZ,ZB1,([1],[0]))

ZB2 = np.tensordot(dX,ZB,([1],[0]))
ZB2 = np.tensordot(dX,ZB2,([1],[0]))
ZB2 = np.tensordot(dX,ZB2,([1],[0]))
ZB2 = np.tensordot(dId,ZB2,([1],[0]))

In [9]:
u = np.kron(Id,Id).reshape([2,2,4])

T1 = np.tensordot(u,u,([1],[0]))
T1 = np.tensordot(T1,u,([2],[0]))
T1 = np.tensordot(T1,u,([3],[0]))
T1 = T1.transpose([5,2,1,3,4,0])



In [10]:
M1 = np.tensordot(ZA,T1,([3],[2]))
M1 = M1.transpose([0,3,1,4,2,5,6,7])
M1 = M1.reshape([16,16,4,4,2,2]) 

M2 = np.tensordot(T1,ZB,([3],[2]))
M2 = M2.transpose([0,5,1,6,2,7,3,4])
M2 = M2.reshape([16,16,4,4,2,2]) 

In [11]:
trans1 = np.trace(M1,axis1= 4, axis2=5)
trans2 = np.trace(M2,axis1= 4, axis2=5)

In [26]:
MHA = np.zeros([2,2,2,2,16,16])
MHA[0,0,0,0] = np.kron(dId,dId)
MHA[1,1,1,1] = np.kron(dId,dId)
MHA[0,1,1,0] = HA


MHB = np.zeros([2,2,2,2,16,16])
MHB[0,0,0,0] = np.kron(dId,dId)
MHB[1,1,1,1] = np.kron(dId,dId)
MHB[0,1,1,0] = HB

In [27]:
MHA = MHA.reshape([2]*12)
MHA = MHA.transpose([4,0,8,5,1,9,6,2,10,7,3,11])
MHA = MHA.reshape([8]*4)

MHB = MHB.reshape([2]*12)
MHB = MHB.transpose([4,0,8,5,1,9,6,2,10,7,3,11])
MHB = MHB.reshape([8]*4)

In [18]:
T2 = np.kron(T1,dId).reshape([2]*14)
T2 = T2.transpose([0,10,1,2,12,3,4,13,5,6,11,7,8,9])
T2 = T2.reshape([8,8,8,8,2,2])

In [21]:
MH1 = np.tensordot(MHA,T2,([3],[2]))
MH1 = MH1.transpose([0,3,1,4,2,5,6,7])
MH1 = MH1.reshape([64,64,8,8,2,2]) 

MH2 = np.tensordot(T2,MHB,([3],[2]))
MH2 = MH2.transpose([0,5,1,6,2,7,3,4])
MH2 = MH2.reshape([64,64,8,8,2,2]) 

In [695]:
X = np.array([[0,1],[1,0]])
Id = np.eye(2)

X12 = np.zeros([1,1,4,4])
X12[0,0,:,:] = np.kron(X,X)

Z12 = np.zeros([1,1,4,4])
Z12[0,0,:,:] = np.kron(Z,Z)


X1 = np.zeros([1,1,4,4])
X1[0,0,:,:] = np.kron(X,Id)

Z1 = np.zeros([1,1,4,4])
Z1[0,0,:,:] = np.kron(Z,Id)


XZ12 = np.zeros([1,1,4,4])
XZ12[0,0,:,:] = np.kron(X@Z,X@Z)


MPOdX = iMPS.iMPO()
MPOdX.construct_from_tensor_list([X12,X12])

MPOX = iMPS.iMPO()
MPOX.construct_from_tensor_list([X1,X1])

MPOZ = iMPS.iMPO()
MPOZ.construct_from_tensor_list([Z1,Z1])


MPOdZ = iMPS.iMPO()
MPOdZ.construct_from_tensor_list([Z12,Z12])


MPOdXZ = iMPS.iMPO()
MPOdXZ.construct_from_tensor_list([XZ12,XZ12])



In [724]:
lam1ss=[]
lam2ss=[]
lam12ss=[]

dims=[]


Z = np.array([[1,0],[0,-1]])
B = np.zeros([1,1,4])
A = np.array([[0.4,0.1],[0.2,0.3]])
#B[0,0] =(linalg.expm(0.1*np.kron(Z,Z))).reshape([16,])
B[0,0] =(linalg.expm(0.1*A)).reshape([4,])
MPS = iMPS.iMPS()
MPS.svd_threshold =1e-10
MPS.max_bond = 15
MPS.construct_from_tensor_list([B,B])

rng = np.arange(0.,.01,0.05)
for p1 in np.arange(0.,.01,0.05): 
    lamdXs=[]
    lamdZs=[]
    lamdXZs=[]

    
    for p2 in rng:
        #trans = funcs.single_trans_qc2(p1=p1,channel1='x_flip',p2=p2,channel2='z_flip')
        #trans = funcs.single_trans_2layers(p1=0,channel1='x_flip',p2=p2,channel2='x_flip')
        #trans = funcs.single_trans_2layers_swap(p1=p1,p2=p2,channel='x_flip')
        MPO = iMPS.iMPO()
        MPO.construct_from_tensor_list([trans1,trans2])

        MPS_power2 = iMPS.MPS_power_method_twosite(MPS,MPO,max_bond=15)
        MPS_power2.update(40,1e-6,1e-6)  
        
        dims.append(MPS_power2.MPS2.chi[0])
        
        strap = iMPS.strap(MPS_power2.MPS2,MPOdX,MPS_power2.MPS2)
        lamdX = strap.calculate_eig()
        
        strap = iMPS.strap(MPS_power2.MPS2,MPOX,MPS_power2.MPS2)
        lamX = strap.calculate_eig()
        strap = iMPS.strap(MPS_power2.MPS2,MPOZ,MPS_power2.MPS2)
        lamZ = strap.calculate_eig()
        
        strap = iMPS.strap(MPS_power2.MPS2,MPOdZ,MPS_power2.MPS2)
        lamdZ = strap.calculate_eig()
        
        strap = iMPS.strap(MPS_power2.MPS2,MPOdXZ,MPS_power2.MPS2)
        lamdXZ = strap.calculate_eig()
        
        
    

        lamdXs.append(lamdX)
        lamdZs.append(lamdZ)
        lamdXZs.append(lamdXZ)

        print(p2)



0.0


In [725]:
MPS_power2.MPS2.B

[array([[[-0.70710678+0.j,  0.        +0.j,  0.        +0.j, -0.70710678+0.j]]]),
 array([[[-0.70710678+0.j,  0.        +0.j,  0.        +0.j, -0.70710678+0.j]]])]

In [726]:
print(lamdX,lamdZ, lamdXZ)

(0.9999999999999999+0j) (0.9999999999999999+0j) (0.9999999999999999+0j)


In [727]:
print(lamX,lamZ)

0j 0j
